# Machine Learning Capstone Project

## TalkingData AdTracking Fraud Detection Challenge

----------------------------------

### 1. Project Overview

With the latest web technologies there are many new avenues that are opened for the mankind. There are Millions of companies across the Globe that are providing varied services including Domain Registration, Online Gaming, Web Hosting, Cab Booking, Food Ordering, Health Consultation, Online Advertising and lot more. Many of these services makes our life very easy but at the same time we can easily become a victim of a fraud if we are not careful. One of the frauds that is related to Online Advertising is "Click Fraud". Click fraud is a type of fraud that occurs on the Internet in Pay-Per-Click (PPC), Pay-Per-Action (PPA) or Cost-Per-Activity (CPA) online advertising. In this type of advertising, the websites owners that post the ads are paid an amount of money determined by how many visitors to the sites or mobile app click on the ads. Fraud occurs when a person, automated script or computer program imitates a legitimate user clicking on such an ad without having an actual interest in the target of the ad's link.

### 2. Problem Statement

One of the major areas of "Click Fraud" is in the area of mobile ad channels where automated scripts may click mobile ads or download a mobile app without a real reason. The problem that we will be solving here is to predict if a user click is genuine or fraudulent. With over 1 billion smart mobile devices in active use every month, China is the largest mobile market in the world and therefore suffers from huge volumes of fraudulent traffic.

TalkingData, China’s largest independent big data service platform, covers over 70% of active mobile devices nationwide. They handle 3 billion clicks per day, of which 90% are potentially fraudulent. Their current approach to prevent click fraud for app developers is to measure the journey of a user’s click across their portfolio, and flag IP addresses who produce lots of clicks, but never end up installing apps.

So, here we have to build an algorithm that predicts whether a user will download an app after clicking a mobile app Ad.

### 3. Datasets and Inputs

To build an algorithm that predicts whether a user will download an app after clicking a mobile app ad TalkingData has provided us a generous dataset covering approximately 200 million clicks over 4 days.

**File descriptions**

- train.csv - The training set consisting of approximately 200 Million rows
- train_sample.csv - 100,000 randomly-selected rows of training data
- test.csv - the testing set consisting of approximately 20 Million rows

**Data fields**

Each row of the training data contains a click record, with the following features:

- ip: IP Address of click.
- app: App id for marketing.
- device: Device type id of user mobile phone (e.g., iPhone 6 Plus, iPhone 7, Huawei mate 7, etc.)
- os: OS version id of user mobile phone
- channel: Channel id of mobile ad publisher
- click_time: Timestamp of click (UTC)
- attributed_time: If user download the app for after clicking an ad, this is the time of the app download
- is_attributed: The target that is to be predicted, indicating the app was downloaded

### 4. Evaluation Metrics

As the submissions are evaluated on area under the ROC curve between the predicted probability and the observed target we will also use the same metric.

A receiver operating characteristic (ROC), or simply ROC curve, is a graphical plot which illustrates the performance of a binary classifier system as its discrimination threshold is varied. It is created by plotting the fraction of true positives out of the positives (TPR = true positive rate) vs. the fraction of false positives out of the negatives (FPR = false positive rate), at various threshold settings. TPR is also known as sensitivity, and FPR is one minus the specificity or true negative rate.

### 5. Load Required Libraries

In [12]:
# Load required libraries
import os
import subprocess
import gc

import sqlite3
import zipfile

import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

### 6. Train and Test File Details

In [5]:
# Read the Train and Test files
zf_train = zipfile.ZipFile('train.csv.zip', mode='r')
zf_test = zipfile.ZipFile('test.csv.zip', mode='r')

In [6]:
# Print details of the Train csv file.
for info in zf_train.infolist():
    print("File Name         -> {}".format(info.filename))
    print("Compressed Size   -> {:.2f} {}".format(info.compress_size/(1024*1024), "MB"))
    print("UnCompressed Size -> {:.2f} {}".format(info.file_size/(1024*1024), "MB"))

File Name         -> mnt/ssd/kaggle-talkingdata2/competition_files/train.csv
Compressed Size   -> 1238.43 MB
UnCompressed Size -> 7188.46 MB


In [7]:
# Print details of the Test csv file.
for info in zf_test.infolist():
    print("File Name         -> {}".format(info.filename))
    print("Compressed Size   -> {:.2f} {}".format(info.compress_size/(1024*1024), "MB"))
    print("UnCompressed Size -> {:.2f} {}".format(info.file_size/(1024*1024), "MB"))

File Name         -> test.csv
Compressed Size   -> 161.93 MB
UnCompressed Size -> 823.28 MB


As we see above, we have a very huge train file that has an UnCompressed Size of around 7.2GB and compressed file is around 1.2GB. Also the test file is around 823MB uncompressed and after compression comes to around 162MB.

In [15]:
# Print records in train.csv
with open('train.csv') as f:
    size=len([0 for _ in f])
    print("Records in train.csv => {}".format(size))

Records in train.csv => 184903891


In [16]:
# Print records in test.csv
with open('test.csv') as f:
    size=len([0 for _ in f])
    print("Records in test.csv => {}".format(size))

Records in test.csv => 18790470


As can be seen from the above output, we have around **184.9 Million** rows in the train set and **18.8 Million** rows in the test set.

As it is very difficult to work on the complete training data, we will use the last **25 Million** rows from the training set and build a model.

### 7. Analyze Train_Sample Data

Before we go ahead and read the training data we will take a look at the "train_sample.csv" file provided by the organizers.

In [20]:
train_sample = pd.read_csv("train_sample.csv")
train_sample.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [21]:
train_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null int64
app                100000 non-null int64
device             100000 non-null int64
os                 100000 non-null int64
channel            100000 non-null int64
click_time         100000 non-null object
attributed_time    227 non-null object
is_attributed      100000 non-null int64
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


As the data is huge we could do a simple optimization to effectively use our memory. We know that Python uses its best of knowledge to decide on the data types and might allocate a higher size data type than really required, say int64 instead of unit16.

A simple step that we will be doing here is to **sqlite3** to build a database file from the train.csv and analyze the data in each of the coulmns. With this information we will be able to decide on the best data type for each of the features.

```
con = sqlite3.connect("talkingdata_train.db")  # Opens file if exists, else creates file
cur = con.cursor()  # This object lets us actually send messages to our DB and receive results

sql = "SELECT sql FROM sqlite_master WHERE name='test_data'"
cur.execute(sql)

if not cur.fetchall():
    for chunk in pd.read_csv("train.csv", chunksize=5000):
        chunk.to_sql(name="train_data", con=con, if_exists="append", index=False)  #"name" is name of table
        gc.collect()

# Below SQL queries can be used to get the min and max values for each of the features


Below we will print the max and min values for the ip column in the train_data table. The actual results returned for this query on the complete train data is (1, 364778). This values will help us to choose a datatype for this column when loading the data in to a pandas DataFrame. So, for ip we can choose uint32

sql = "select min(ip), max(ip) from train_data"
cur.execute(sql)
cur.fetchall()

Below we will print the max and min values for the app column in the train_data table. The actul results returned for this query on the complete train data is (0, 768). This values will help us to choose a datatype for this column when loading the data in to a pandas DataFrame. So, for app we can choose uint16

sql = "select min(app), max(app) from train_data"
cur.execute(sql)
cur.fetchall()

Below we will print the max and min values for the device column in the train_data table. The actul results returned for this query on the complete train data is (0, 4227). This values will help us to choose a datatype for this column when loading the data in to a pandas DataFrame. So, for device we can choose uint16

sql = "select min(device), max(device) from train_data"
cur.execute(sql)
cur.fetchall()

Below we will print the max and min values for the os column in the train_data table. The actul results returned for this query on the complete train data is (0, 956). This values will help us to choose a datatype for this column when loading the data in to a pandas DataFrame. So, for os we can choose uint16

sql = "select min(os), max(os) from train_data"
cur.execute(sql)
cur.fetchall()

Below we will print the max and min values for the channel column in the train_data table. The actul results returned for this query on the complete train data is (0, 500). This values will help us to choose a datatype for this column when loading the data in to a pandas DataFrame. So, for channel we can choose uint16

sql = "select min(channel), max(channel) from train_data"
cur.execute(sql)
cur.fetchall()

Below we will print the max and min values for the is_attributed column in the train_data table. The actul results returned for this query on the complete train data is (0, 1). This values will help us to choose a datatype for this column when loading the data in to a pandas DataFrame. So, for is_attributed we can choose uint8

sql = "select min(is_attributed), max(is_attributed) from train_data"
cur.execute(sql)
cur.fetchall()

```

Below we will see what percentage of data is attributed.

```
Below we will get the records where the value of is_attributed is 1. For the actual data we can see that this value is 456846 which means that we have only 0.247 % of records where is_attributed is 1.

sql = "select count(*) from train_data where is_attributed=1"
cur.execute(sql)
cur.fetchall()
```

As can be seen from the above information we have a very highly imbalanced data set.

In [22]:
# Data types of each of the features are chosen based on the value range instead of relying on the python interpreter.
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

### 8. Read Train and Test Data

In [27]:
# Read the last 25 Million records from training data
train_df = pd.read_csv("train.csv", skiprows=range(1,159903891), nrows=25000000, dtype=dtypes)
train_df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,33748,18,1,10,134,2017-11-09 08:15:22,NaN,0
1,26810,3,1,25,489,2017-11-09 08:15:22,NaN,0
2,105587,21,1,13,128,2017-11-09 08:15:22,NaN,0
3,3542,2,1,13,435,2017-11-09 08:15:22,NaN,0
4,124339,3,1,41,211,2017-11-09 08:15:22,NaN,0


**attributed_time** is set when the **is_attributed** field is True. This is the time when an App was actually downloaded after an Ad was clicked.

In [26]:
# Read the records from test data
test_df = pd.read_csv("test.csv", dtype=dtypes)
test_df.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [29]:
train_df.nunique()

ip                 91647
app                  435
device              1905
os                   396
channel              182
click_time         27879
attributed_time    24540
is_attributed          2
dtype: int64